
# Interpreter v1

Implement an interpreter for a small subset of scheme with the following features in Python.
```
    Constant literals - only integers
    variable reference
    if/conditionals
    define
    procedure call - only primitive procedures (+, *, >, <, =)
```


In [2]:
(import "builtins")
(define python builtins)
(import "math")

(math)

In [3]:
(define apply-in-underlying-scheme apply)

In [4]:

; eval

(define (eval exp env)
  ;; (display (format "exp** ~a \n" exp))
  (cond ((self-evaluating? exp)
             exp)
        ((variable? exp)
             (lookup-variable-value exp env))
        ((definition? exp)
             (eval-definition exp env))
        ((if? exp)
             (eval-if exp env))
        ((lambda? exp)
             (eval-lambda exp env))
        ((application? exp)
             (apply (eval (operator exp) env)
                    (list-of-values (operands exp) env) env))
        (else
             (error "Unknown expression type: EVAL" exp))))


(define (list-of-values exps env)
  (map (lambda (e) (eval e env))
       exps))

(define operator car)
(define operands cdr)

;; types

(define (self-evaluating? exp) 
   (number? exp))
(define (variable? exp) (symbol? exp))
(define (definition? exp) (tagged-list? exp 'define))
(define (if? exp) (tagged-list? exp 'if))
(define (application? exp) (pair? exp))

(define (tagged-list? exp tag)
  (and (pair? exp) 
       (eq? (car exp) tag)))




;; if

(define (eval-if exp env)
  (let ((t (eval (if-test exp) env)))
    (if t (eval (if-consequence exp) env) (eval (if-alternative exp) env))))

(define if-test cadr)
(define if-consequence caddr)
(define if-alternative cadddr)

;; definition

(define (eval-definition exp env)
  
    (let ((var (definition-var exp)))
        (cond ((variable? var) 
                  (let ((val (eval (definition-value exp) env)))
                       (set-variable! var val env)))

              ((list? var)
                   (let ((params (cdr var))
                         (body (definition-value exp)))
                       (set-variable! (car var) (make-procedure params body None) env)))

              (else (error eval-definition "invalid definition decalaration")))))

(define definition-var cadr)
(define definition-value caddr)





**Task:**

Add support for lambda and function application.
```
(define (apply proc arguments)
  (cond ((primitive-procedure? proc)
         (apply-in-underlying-scheme proc arguments))
        ((compound-procedure? proc)
         ...
         )
        (else
         (error "unknown procedure" proc))))



(define (primitive-procedure? proc) (tagged-list? 'primitive))
(define (compound-procedure? proc) (tagged-list? 'compound-procedure))

(define (make-procedure params body env)
  ...)

(define (procedure-params proc) ...)
(define (procedure-body proc) ...)
(define (procedure-env proc) ...)

(define (lambda? exp) ...)

(define lambda-params cadr)
(define lambda-body caddr)

(define (eval-lambda exp env)
  (make-procedure (lambda-params exp)
                  (lambda-body exp)
                  env))
```
How to represent the env as chain of frames.


In [5]:

;; procedure

(define (primitive-procedure? proc) (tagged-list? proc 'primitive))
(define (compound-procedure? proc) (tagged-list? proc 'compound-procedure))

(define (make-procedure params body env)
  (list 'compound-procedure params body env)
)

(define (procedure-params proc) 
    (cadr proc))
(define (procedure-body proc)
    (caddr proc))
(define (procedure-env proc)
    (cadddr proc))

;; lambda

(define (lambda? exp) 
      (eq? (car exp) 'lambda))

(define lambda-params cadr)
(define lambda-body caddr)

(define (eval-lambda exp env)
  (make-procedure (lambda-params exp)
                  (lambda-body exp)
                  env))

In [6]:
;; apply

(define (apply proc arguments env)
  ;; (display (format "*apply* ~a ~a \n" proc arguments))
  (cond ((primitive-procedure? proc)
             (apply-in-underlying-scheme (cadr proc) arguments))
        ((compound-procedure? proc)
             (apply-compound-procedure proc arguments env))
        ((compound-procedure? proc)
             (apply-compound-procedure proc arguments env))
        (else
             (error "unknown procedure" proc))))

(define (apply-compound-procedure proc args env) 
        ;; (display (format "apply-compund* ~a ~a \n" proc args))
        (let (
                (proc-params (procedure-params proc))
                (proc-body (procedure-body proc))
                (proc-env (procedure-env proc)))

                (let ((newenv (extend-env proc-params args env)))
                    (eval proc-body newenv))))

### Env

In [7]:

;; frame

(define (make-frame variables values)
   (cons variables values))

(define frame-variables car)
(define frame-values cdr)

(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var (car frame)))
  (set-cdr! frame (cons val (cdr frame))))

(define (lookup-frame-var var frame)
  (define (scan vars vals)
    (cond ((null? vars) #f)
          ((eq? (car vars) var) (car vals))
          (else (scan (cdr vars) (cdr vals)))))
  (scan (car frame) (cdr frame)))


In [8]:
;; env

(define (make-env)
  '())

(define (empty? env) 
  (or (null? env) (eq? env None)))

(define (extend-env params args env) 
  (let ((new-frame (make-frame params args))) 
    (cons new-frame env)
    ))

(define (frame-env env) (car env))

(define (parent env)
  (if (null? (cdr env))
      None
      (cdr env)))

(define (lookup-env-var var env)
  (cond ((empty? env) #f)
        (else 
             (let ((lookupval (lookup-frame-var var (frame-env env))))
                   (cond ((not lookupval) 
                              (lookup-env-var var (parent env)))
                         (else lookupval))))
  ))

(define lookup-variable-value lookup-env-var)


(define (set-variable! var val env) 
      (if (empty? env)
          None
          (add-binding-to-frame! var val (frame-env env))))

In [33]:
;; primitives

(define primitives 
  (list
   (list '+ (list 'primitive +))
   (list '* (list 'primitive *))
   (list '- (list 'primitive -))
   (list '/ (list 'primitive /))
   (list '% (list 'primitive %))
   (list '> (list 'primitive >))
   (list '< (list 'primitive <))
   (list '<= (list 'primitive <=))
   (list 'eq? (list 'primitive eq?))))

(define primitive-vars (map car primitives))
(define primitive-vals (map cadr primitives))

(define (setup-env)
  (let ((newenv (make-env)))
    (extend-env primitive-vars primitive-vals newenv)
    ))

(define (top-frame env)
  (cadr env))

In [34]:
;; test env

(define x (setup-env))
(define x1 (extend-env '(x y) '(1 2) x))
(define x2 (extend-env '(w z) '(3 4) x1))

(lookup-env-var 'f x2)

#f

In [35]:
;; setup env

(define the-global-env (setup-env))
(define env the-global-env)


In [36]:
(eval '1 env)

1

In [37]:
(eval '(define x 5) env)

In [38]:
(eval '(+ x 1) env)

6

In [39]:
(eval 'x env)

5

In [40]:
env

(((x + * - / % > < <= eq?) 5 (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>)))

In [41]:
(eval '(define (add y) (+ y y)) env)

In [42]:
env

(((add x + * - / % > < <= eq?) (compound-procedure (y) (+ y y) None) 5 (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>) (primitive #<procedure>)))

In [43]:
(eval '(add 2) env)

4

In [44]:
(eval '(define (square y) (* y y)) env)

In [45]:
(eval '(square 5) env)

25

In [46]:
(eval '((lambda (x) (+ x x)) 2) env)

4

In [47]:
(eval '(define t (lambda (x) (+ x x))) env)

In [48]:
(eval '(t 5) env)

10

In [49]:
(eval '(define (factorial n)
             (if (eq? n 1) 
                 1
                 (* n (factorial (- n 1))))) env)

In [50]:
(eval '(factorial 5) env)

120

In [51]:
(eval '(if (> (t 5) 9) 1 0) env)

1

In [52]:
(eval '(square (factorial 5)) env)

14400

In [53]:
(eval '(define circle-area (lambda (r) (* 3.14159 (* r r)))) env)

In [54]:
(eval '(circle-area 3) env)

28.27431

In [55]:
(eval '(define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1)))))) env)

In [56]:
(eval '(fact 10) env)

3628800